1. Add the "OwnerParcel" layer from the parcels GDB for the city
2. Add the zoning layer

In [24]:
city = 'Billings'
zone_field = 'ZONE_CODE'
source_parcels = r'shp\Yellowstone_GDB\Yellowstone_Parcels.gdb\OwnerParcel'
source_zoning = r'shp\zoning.gdb\billings'

import os
import shutil
gdb = os.path.join('shp', city + '.gdb')
spatial_join_layer = "OwnerParcel_SpatialJoin_" + city

In [17]:
# if baseline GDB exists, delete it
if os.path.isdir(gdb):
    shutil.rmtree(gdb)

arcpy.management.CreateFileGDB('shp', city)

<Result 'shp\\Billings.gdb'>

In [18]:
arcpy.conversion.FeatureClassToFeatureClass(
    source_parcels,
    gdb,
    "OwnerParcel"
)

<Result 'shp\\Billings.gdb\\OwnerParcel'>

In [19]:
arcpy.conversion.FeatureClassToFeatureClass(
    source_zoning,
    gdb,
    "Zoning"
)

<Result 'shp\\Billings.gdb\\Zoning'>

In [3]:
arcpy.analysis.SpatialJoin(
    "OwnerParcel", 
    "Zoning", 
    os.path.join(gdb, spatial_join_layer), 
    "JOIN_ONE_TO_ONE", 
    "KEEP_COMMON", 
    None, 
    "HAVE_THEIR_CENTER_IN")

<Result 'shp\\Billings.gdb\\OwnerParcel_SpatialJoin_Billings'>

In [8]:
arcpy.management.SelectLayerByAttribute(
    spatial_join_layer, 
    "NEW_SELECTION", 
    """
    ZONE_CODE IN ('N1', 'N2', 'N3', 'NX1', 'NX2', 'NX3', 'RMH') 
    AND PARCELID IS NOT NULL 
    AND PropType NOT IN (
        '',
        'VR - Vacant Land Rural', 
        'CA - Centrally Assessed', 
        'VU - Vacant Land Urban',
        'FARM_U - Farmstead - Urban', 
        'NVS - Non-Valued with Specials', 
        'RV_PARK - RV Park', 
        'GRAVEL - Gravel Pit', 
        'GOLF - Golf Course',
        'EP_PART - Partial Exempt Property', 
        'CN - Centrally Assessed Non-Valued Property', 
        'NV - Non-Valued Property', 'FARM_R - Farmstead - Rural', 
        'VAC_U - Vacant Land - Urban', 
        'EP - Exempt Property', 'VAC_R - Vacant Land - Rural'
    )
    """, 
    "NON_INVERT"
)

id,value
0,a Layer object
1,37971


In [9]:
arcpy.conversion.FeatureClassToFeatureClass(
    spatial_join_layer, 
    gdb, 
    "residential_parcels"
)
# then rename to residential_parcels

<Result 'shp\\Billings.gdb\\residential_parcels'>

In [10]:
arcpy.management.CalculateGeometryAttributes(
    "residential_parcels", 
    "lot_size AREA", 
    '', 
    "SQUARE_FEET_US", 
    None, 
    "SAME_AS_INPUT"
)

<Result 'residential_parcels'>

In [11]:
arcpy.management.Project(
    "residential_parcels", 
    os.path.join(gdb, 'parcels_ft'), 
    'PROJCS["NAD_1983_StatePlane_Montana_FIPS_2500_Feet",GEOGCS["GCS_North_American_1983",DATUM["D_North_American_1983",SPHEROID["GRS_1980",6378137.0,298.257222101]],PRIMEM["Greenwich",0.0],UNIT["Degree",0.0174532925199433]],PROJECTION["Lambert_Conformal_Conic"],PARAMETER["False_Easting",1968500.0],PARAMETER["False_Northing",0.0],PARAMETER["Central_Meridian",-109.5],PARAMETER["Standard_Parallel_1",45.0],PARAMETER["Standard_Parallel_2",49.0],PARAMETER["Latitude_Of_Origin",44.25],UNIT["Foot_US",0.3048006096012192]]', "'WGS_1984_(ITRF08)_To_NAD_1983_2011 + WGS_1984_(ITRF00)_To_NAD_1983'", 'PROJCS["NAD_1983_2011_StatePlane_Montana_FIPS_2500",GEOGCS["GCS_NAD_1983_2011",DATUM["D_NAD_1983_2011",SPHEROID["GRS_1980",6378137.0,298.257222101]],PRIMEM["Greenwich",0.0],UNIT["Degree",0.0174532925199433]],PROJECTION["Lambert_Conformal_Conic"],PARAMETER["False_Easting",600000.0],PARAMETER["False_Northing",0.0],PARAMETER["Central_Meridian",-109.5],PARAMETER["Standard_Parallel_1",45.0],PARAMETER["Standard_Parallel_2",49.0],PARAMETER["Latitude_Of_Origin",44.25],UNIT["Meter",1.0]]', 
    "NO_PRESERVE_SHAPE", 
    None, 
    "NO_VERTICAL"
)

<Result 'shp\\Billings.gdb\\parcels_ft'>

In [12]:
arcpy.management.MinimumBoundingGeometry(
    os.path.join(gdb, 'parcels_ft'), 
    os.path.join(gdb, 'parcels_mbb'), 
    "RECTANGLE_BY_WIDTH", 
    "NONE", 
    None, 
    "MBG_FIELDS"
)

<Result 'shp\\Billings.gdb\\parcels_mbb'>

In [13]:
arcpy.management.JoinField(
    "residential_parcels", 
    'PARCELID', 
    os.path.join(gdb, 'parcels_mbb'), 
    'PARCELID', 
    'MBG_Width;MBG_Length'
)

<Result 'residential_parcels'>

In [14]:
arcpy.management.CalculateField(
    "residential_parcels", 
    "city", 
    '"' + city + '"'
)

<Result 'residential_parcels'>

In [25]:
arcpy.management.CalculateField(
    "residential_parcels", 
    "zoning", 
    "!" + zone_field + "!", 
    "PYTHON3"
)

<Result 'residential_parcels'>

In [16]:
arcpy.management.CalculateField(
    "residential_parcels", 
    "lot_width", 
    "!MBG_Width!", 
    "PYTHON3", 
    None, 
    "SHORT"
)

<Result 'residential_parcels'>

In [26]:
# units allowed by underlying zoning
codeblock = """
def eval_zoning(zone):
    if zone in ['N3', 'RMH']:
        return 1
    elif zone in ['N1', 'N2']:
        return 2
    elif zone in ['NX1']:
        return 4
    elif zone in ['NX2', 'NX3']:
        return 5
    else:
        return 0
"""

arcpy.management.CalculateField(
    "residential_parcels", 
    "zoning_allows", 
    "eval_zoning(!zoning!)", 
    "PYTHON3", 
    codeblock, 
    "SHORT"
)

<Result 'residential_parcels'>

In [27]:
codeblock = """
def eval_zoning(zone, lot_size):
    return 5
"""

arcpy.management.CalculateField(
    "residential_parcels", 
    "lot_size_allows", 
    "eval_zoning(!zoning!, !lot_size!)", 
    "PYTHON3", 
    codeblock, 
    "SHORT"
)

ExecuteError: ERROR 160706: Cannot acquire a lock.
Failed to execute (CalculateField).


In [31]:
codeblock = """
def eval_zoning(zone, lot_width):
    if zone == 'N1' and lot_width >= 20:
        return 2
    elif zone == 'N2':
        if lot_width >= 50:
            return 2
        else:
            return 0
    elif zone == 'N3':
        if lot_width >= 65:
            return 1
        else:
            return 0
    elif zone == 'NX1':
        if lot_width >= 50:
            return 4
        elif lot_width >= 20:
            return 2
        else:
            return 0
    elif zone == 'NX2':
        if lot_width >= 50:
            return 5
        elif lot_width >= 20:
            return 2
        else:
            return 0
    elif zone == 'NX3':
        if lot_width >= 50:
            return 5
        else:
            return 0
    else:
        return 0
"""

arcpy.management.CalculateField(
    "residential_parcels", 
    "lot_width_allows", 
    "eval_zoning(!zoning!, !lot_width!)", 
    "PYTHON3", 
    codeblock, 
    "SHORT"
)

<Result 'residential_parcels'>

In [ ]:
# note - had to manually delete one parcel with NULL lot width

In [32]:
codeblock = """
def units(zone, size, width):
    return min(zone, size, width)
"""
arcpy.management.CalculateField(
    "residential_parcels", 
    "units_allowed", 
    "units(!zoning_allows!, !lot_size_allows!, !lot_width_allows!)",
    "PYTHON3",
    codeblock,
    "SHORT"
)

<Result 'residential_parcels'>

In [37]:
codeblock = """
def categorize(zone):
    if zone in ['N1', 'N2', 'NX1', 'NX2', 'NX3']:
        return "Multifamily"
#    elif zone in []:
#        return "Penalized Multifamily"
    elif zone in ['N3', 'RMH']:
        return "Single-Family"
    else:
        return "Unknown"
"""
arcpy.management.CalculateField(
    "residential_parcels", 
    "category", 
    "categorize(!zoning!)",
    "PYTHON3",
    codeblock,
    "TEXT"
)

<Result 'residential_parcels'>

In [38]:
arcpy.conversion.FeatureClassToFeatureClass(
    "residential_parcels", 
    gdb, 
    city.lower() + "_residential_parcels"
)

<Result 'shp\\Billings.gdb\\billings_residential_parcels'>

In [39]:
working_layer = os.path.join(gdb, city.lower() + "_residential_parcels")

fields = [f.name for f in arcpy.ListFields(working_layer)]
keep_fields = ['OBJECTID', 'SHAPE', 'Shape', 'Shape_Area', 'Shape_Length',
               'PARCELID', 'lot_size',
               'city', 'zoning', 'lot_width', 'zoning_allows', 
               'lot_size_allows', 'lot_width_allows',
               'units_allowed', 'category']
delete_fields = [field for field in fields if not field in keep_fields]
arcpy.management.DeleteField(working_layer, delete_fields)


<Result 'shp\\Billings.gdb\\billings_residential_parcels'>